In [1]:
# !pip install -q transformers 
# !pip install -q peft
# !pip install -q evaluate
# !pip install -q wandb

In [2]:
# from datasets import load_dataset
# ds = load_dataset("parquet", data_files="data/mmimdb_merged.parquet", split="train") 

C:\Users\Utilisateur\OneDrive - Groupe ESAIP\Bureau\Cetic_code\pythonProject1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# !python.exe -m pip install --upgrade pip

In [4]:
# img = ds[0]["images"]          # -> objet PIL.Image.Image
# img

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=265x475>]

In [5]:
# len(ds[0]["images"])

1

In [6]:
# from PIL import Image
# Image.MAX_IMAGE_PIXELS = 175_000_000
# ds_single = ds.map(
#     lambda ex: {"image": ex["images"][0]},
#     remove_columns=["images"]
# )
# print(ds_single.features)


{'messages': List({'role': Value('string'), 'content': Value('string')}), 'image': Image(mode=None, decode=True)}


In [7]:
# print(ds_single[302]['messages'])

[{'role': 'user', 'content': "Given the movie poster and the corresponding plot of the movie, choose the appropriategenres from the following comma separated genres: drama, comedy, romance, thriller, crime, action, adventure,horror, documentry, mystery, sci-fi, fantasy, family, biography, war, history, music, animation, musical,western, sport, short, film-noir.\nPlot: In the early 1600's, Countess Elizabeth Bathory slaughtered more than 600 young women, believing if she bathed in the blood of virgins that she would stay young and beautiful forever. Still alive today, she's found a perfect hunting ground for her 'botox' as an abstinence educator in conservative America, and the young ladies of San Griento High are poised to be her next victims. But will her unholy ritual finally be stopped by Leah Ratliff, a feminist blogger and ambitious reporter for the school paper?\nNote that a movie can belong to more than one genres,provide all the suitable genres separated by commas. Answer:"}, {

In [ ]:
# ds_single[302]['image']

In [12]:
# def flatten_text_only(example):
# 
#     msgs = example.get("messages", []) or []
#     assistant_msgs = [m.get("content", "") for m in msgs if m.get("role") == "assistant"]
# 
#     answer = assistant_msgs[-1] if assistant_msgs else ""
# 
#     return {
#         "answer": answer,
#         "images": example['image']
#     }
# ds_flat = ds_single.map(flatten_text_only, remove_columns=ds_single.column_names)

Map: 100%|██████████| 15552/15552 [27:02<00:00,  9.58 examples/s]  


In [8]:
from datasets import load_from_disk
import numpy as np

ds_flat = load_from_disk(r"data/ds_flat")

# 1) Vocabulaire des genres (unique & trié)
all_genres = set()
for ans in ds_flat['answer']:
    if ans is None:
        continue
    for g in ans.split(','):
        g = g.strip()
        if g:
            all_genres.add(g)

genres = sorted(all_genres)  # doit faire 23
print("Nb de genres:", len(genres))
print(genres)

assert len(genres) == 26, "Le nombre de genres n'est pas 26. Vérifie tes données."

genre2id = {g:i for i, g in enumerate(genres)}

# 2) Fonction de mapping -> labels_idx + labels (multi-hot)
def encode_labels(example):
    # Parse propre de la réponse
    raw = example.get("answer") or ""
    items = [g.strip() for g in raw.split(",") if g.strip()]

    # indices (ignore genre inconnu, si jamais)
    idxs = [genre2id[g] for g in items if g in genre2id]

    # multi-hot
    y = np.zeros(len(genres), dtype=np.float32)
    if idxs:
        y[idxs] = 1.0

    example["labels_idx"] = idxs                   
    example["labels"] = y                          
    return example

ds_flat = ds_flat.map(encode_labels, desc="Ajout des colonnes labels")

# (Optionnel) sauvegarder le vocabulaire pour l'inférence
import json, os
os.makedirs("data/label_vocab", exist_ok=True)
with open("data/label_vocab/genres.json", "w", encoding="utf-8") as f:
    json.dump({"genres": genres}, f, ensure_ascii=False, indent=2)

Nb de genres: 26
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']


Ajout des colonnes labels: 100%|██████████| 15552/15552 [02:45<00:00, 94.21 examples/s] 


Biography, Comedy, Drama, History
[3, 4, 7, 11]


AttributeError: 'list' object has no attribute 'dtype'

In [9]:
# Vérif rapide
print(ds_flat[301]["answer"])
print(ds_flat[301]["labels_idx"])
arr = np.array(ds_flat[301]["labels"], dtype=np.float32)
print(arr.dtype, arr.shape)


Biography, Comedy, Drama, History
[3, 4, 7, 11]
float32 (26,)


In [1]:
from datasets import Dataset, load_from_disk

# # Sauver
# ds_flat.save_to_disk(r"data/ds_flat_labeled")

# Recharger
# Recharger depuis le dossier sauvegardé
ds_flat = load_from_disk("data/ds_flat_labeled")

C:\Users\Utilisateur\OneDrive - Groupe ESAIP\Bureau\Cetic_code\pythonProject1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import sys, torch
print(sys.executable)
print(torch.__version__, torch.version.cuda)
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

C:\Users\Utilisateur\OneDrive - Groupe ESAIP\Bureau\Cetic_code\pythonProject1\.venv\Scripts\python.exe
2.6.0+cu124 12.4
True NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
ds_flat.set_format(type="torch", columns=["labels"])  # + "pixel_values" quand tes images seront prêtes
batch = ds_flat[:8]
print(batch["labels"].dtype, batch["labels"].shape)   # torch.float32, (8, 26)

torch.float32 torch.Size([8, 26])


In [ ]:
# Normalisation
from datasets import load_from_disk, Image as HFImage
import numpy as np
from PIL import Image
import torch

ds_flat = ds_flat.cast_column("images", HFImage(decode=True))  # <-- important

# Normalisation (ImageNet par défaut ; adapte si tu as tes propres stats)
MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def pil_to_np_rgb_noresize(img: Image.Image):
    if img.mode != "RGB":
        img = img.convert("RGB")
    arr = np.asarray(img).astype(np.float32) / 255.0      # HWC
    arr = (arr - MEAN) / STD
    arr = np.transpose(arr, (2, 0, 1))                    # CHW
    return arr

def pick_first_image(value):
    """Accepte PIL.Image, liste de PIL, ou None."""
    if value is None:
        return None
    if isinstance(value, list) and len(value) > 0:
        return value[0]
    return value  # supposé être une PIL.Image

def add_pixel_values(example):
    img = pick_first_image(example["images"])
    if img is None:
        example["pixel_values"] = None
    else:
        example["pixel_values"] = pil_to_np_rgb_noresize(img).astype(np.float32)
    return example

# 2) Ajouter la colonne sans resize
ds_flat = ds_flat.map(add_pixel_values, desc="pixel_values sans resize")

# 3) Collate avec padding dynamique (tailles variables) pour PyTorch
from torch.utils.data import DataLoader
from torch.nn import functional as F

def collate_pad(batch):
    xs, ys = [], []
    for b in batch:
        if b["pixel_values"] is None:
            continue  # skip si pas d'image
        x = torch.from_numpy(b["pixel_values"]) if isinstance(b["pixel_values"], np.ndarray) else b["pixel_values"]
        xs.append(x)                    # (C,H,W)
        ys.append(b["labels"])
    # max H,W du lot
    C = xs[0].shape[0]
    H = max(x.shape[1] for x in xs)
    W = max(x.shape[2] for x in xs)

    x_pad = torch.zeros(len(xs), C, H, W, dtype=torch.float32)
    mask  = torch.zeros(len(xs), 1, H, W, dtype=torch.bool)
    for i, x in enumerate(xs):
        c,h,w = x.shape
        x_pad[i, :, :h, :w] = x
        mask[i, :, :h, :w] = True

    y = torch.stack(ys) if isinstance(ys[0], torch.Tensor) else torch.tensor(ys)
    return {"pixel_values": x_pad, "labels": y, "attention_mask": mask}

loader = DataLoader(ds_flat, batch_size=8, shuffle=True, collate_fn=collate_pad)

# test d’un batch
batch = next(iter(loader))
print(batch["pixel_values"].shape)   # -> [B, 3, Hmax, Wmax]
print(batch["labels"].shape)
print(batch["attention_mask"].shape) # même Hmax, Wmax


pixel_values sans resize:   2%|▏         | 343/15552 [00:06<09:52, 25.67 examples/s] 

CNN part


In [12]:
# step1_multilabel_skeleton.py
from tqdm import tqdm
import os
import random
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch
import numpy as np

Architecture Alexnet :


In [13]:
class Net(nn.Module):
    def __init__(self,num_labels):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 100, 3,padding=1)
        self.pool1 = nn.MaxPool2d(3, 2)
        
        self.conv2 = nn.Conv2d(100, 32, 3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3,padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3,padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv6 = nn.Conv2d(256, 128, 3,padding=1)
        
        self.conv7 = nn.Conv2d(128, 80, 3,padding=1)
        self.bn7   = nn.BatchNorm2d(80)
        self.pool2 = nn.MaxPool2d(3, 2)
        
        #Applatie la matatrice pour le model 
        self.flatten = nn.Flatten(start_dim=1)
        
        #Une couche de model 
        self.fc1 = nn.LazyLinear(80)
        self.bn_fc1 = nn.BatchNorm1d(80)
        
        #pour tester 
        self.out = nn.Linear(80, num_labels)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool2(F.relu(self.bn7(self.conv7(x))))
        x = self.flatten(x)
        x = F.relu(self.bn_fc1(self.fc1(x)))
        logits = self.out(x)
        
        return logits




In [14]:
from torch.utils.data import DataLoader
train_loader = DataLoader(ds_flat, batch_size=32, shuffle=True)

In [15]:
import torch, torch.nn as nn

num_labels = 26
model = Net(num_labels)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for batch in train_loader:
    images = batch["pixel_values"]          # (B,3,H,W)
    targets = batch["labels"].float()       # (B,26)
    logits = model(images)                  # (B,26)
    loss = criterion(logits, targets)
    optimizer.zero_grad(); loss.backward(); optimizer.step()


KeyError: 'pixel_values'

nromalisation à revoir 